In [12]:
import re
from wikimapper import WikiMapper
import rdflib

In [8]:
from templater3 import Templater
documents_to_learn = ['<b> spam and eggs </b>', '<b> ham and spam </b>',
                          '<b> white and black </b>'] # list of documents
template = Templater()
for document in documents_to_learn:
    template.learn(document)

print('Template created:', template._template)


document_to_parse = '<b> yellow and blue </b>'
print('Parsing other document:', template.parse(document_to_parse))

print('Filling the blanks:', template.join(['', 'red', 'orange', '']))

Template created: [None, '<b> ', None, ' and ', None, ' </b>', None]
Parsing other document: ['', 'yellow', 'blue', '']
Filling the blanks: <b> red and orange </b>


In [46]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

def get_subclasses(entity: str) -> 'list[str]':
   subclass_of = []

   query = '''SELECT ?classLabel WHERE {{
   wd:{entity} wdt:P279 ?class.
      SERVICE wikibase:label {{
      bd:serviceParam wikibase:language "en" .
      }}
   }}'''.format(entity=entity)
   sparql.setQuery(query)

   sparql.setReturnFormat(JSON)
   results = sparql.query().convert()
   for result in results['results']['bindings']:
      subclass_of.append(result['classLabel']['value'])
   return subclass_of


In [50]:
lines = open('simplewiki-20211120-lists-1k.tsv').readlines()
mapper = WikiMapper("index_enwiki-latest.db")
for line in lines[:1]:
    file, word_list = line.split('\t')
    print(file)
    word_list = word_list.split('\\n')
    for word in word_list[0:5]:
        entities = re.findall(r'\[\[(.+?)\]\]',word)
        for entity in entities:
                entity = entity.replace(' ', '_')
                id = mapper.title_to_id(entity)
                print(entity, id)
                if id is not None:
                    print(get_subclasses(id))

         
    print()

April
April_1 Q2510
April_Fools'_Day Q80949
April_1 Q2510
Iran Q794
April_2 Q2511
April_2 Q2511
Thailand|Thai None
wikt:conservation|Conservation None
April_2 Q2511
Autism Q1436063



In [10]:
mapper.id_to_titles("Q2403")

['March_15',
 '15_March',
 '15th_March',
 'March_15th',
 'Mar_15',
 '3/15',
 '15/3',
 '15th_of_March']